# Generating Word Embeddings - Lab

## Introduction

In this lab, we'll learn how to generate our own word embeddings by training our own Word2Vec model, and also by building embedding layers right into our Deep Neural Networks!

## Objectives

You will be able to:

* Demonstrate a basic understanding of the architecture of the Word2Vec model
* Demonstrate an understanding of the various tunable parameters of word2vec such as vector size and window size

## Getting Started

In this lab, we'll start by creating our own word embeddings by making use of the Word2Vec Model. Then, we'll move onto building Neural Networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in our model. 

The easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which we can use to begin training immediately. For this lab, we'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.  In this lab, we'll learn how to train a Word2Vec model on the text data to generate word embeddings for them. In the next lab, we'll then use the vectors created by our Word2Vec model to effectively train a classifier to predict the category of news given the headline and description of each article. In this lab, we won't do any classification, although we will learn how to train a Word2Vec model and explore the relationships between different word vectors in our embedding!

Run the cell below to import everything we'll need for this lab. 

In [3]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, we'll import the data. You'll find the data stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a github repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json` function from pandas to read the dataset into a DataFrame. Be sure to also include the parameter `lines=True` when reading in the dataset!

Once you've loaded in the data, inspect the head of the DataFrame to see what our data looks like. 

In [5]:
# News_Category_Dataset_v2.zip
raw_df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
raw_df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


## Preparing the Data

Since we're working with text data, we'll still need to do some basic preprocessing and tokenize our data. You'll notice from the sample of the data above that two different columns contain text data--`headline` and `short_description`. The more text data our Word2Vec model has, the better it will perform. Therefore, we'll want to combine the two columns before tokenizing each comment and training our Word2Vec model. 

In the cell below:

* Create a column called `combined_text` that consists of the data from `df.headline` plus a space character (`' '`) plus the data from `df.short_description`.
* Use the `combined_text` column's `map()` function and pass in `word_tokenize`. Store the result returned in `data`.

In [7]:
raw_df['combined_text'] = raw_df.headline + ' ' +  raw_df.short_description
data = raw_df['combined_text'].map(word_tokenize)

Let's inspect the first 5 items in `data` to see how everything looks. 

In [8]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

You'll notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Recall from our previous lesson on how Word2Vec works that we can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If our window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example we saw).

Now that we've prepared our data, let's train our model and explore a bit!

## Training the Model

We'll start by instantiating a Word2Vec Model from gensim below. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1`.
    * The number of threads to use during training, `workers=4`

In [9]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that we've created our Word2Vec model, we still need to train it on our model. 

In the cell below:

* Call `model.train()` and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which we can find in `model.corpus_count`. 
    * The number of `epochs` we want to train for, which we'll set to `10`

In [10]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55568415, 67352790)

Great! We now have a fully trained model! The word vectors themselves are stored inside of a `Word2VecKeyedVectors` instance, which we'll find stored inside of `model.wv`. For simplicity's sake, let's go ahead and store this inside of the variable `wv` in order to save ourselves some keystrokes down the line. 

In [11]:
wv = model.wv

## Examining Our Word Vectors

Now that we have a trained Word2Vec model, let's go ahead and explore the relationships between some of the words in our corpus! 

One cool thing we can use Word2Vec for is to get the most similar words to a given word. We can do this passing in the word to `wv.most_similar()`. 

In the cell below, let's try getting the most similar word to `'Texas'`.

In [12]:
wv.most_similar('Texas')

[('Maryland', 0.821438193321228),
 ('Ohio', 0.818824052810669),
 ('Oregon', 0.8182888031005859),
 ('Louisiana', 0.7994305491447449),
 ('Michigan', 0.7991020083427429),
 ('Oklahoma', 0.7977007627487183),
 ('Pennsylvania', 0.7934329509735107),
 ('Georgia', 0.7876843214035034),
 ('Illinois', 0.7876551151275635),
 ('Connecticut', 0.7844222784042358)]

Interesting! All of the most similar words are also states. 

We can also get the least similar vectors to a given word by passing in the word to the `most_similar()` function's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [13]:
wv.most_similar(negative='Texas')

[('Parent/Grandparent', 0.4566422700881958),
 ('Headstrong', 0.43431705236434937),
 ('much-vaunted', 0.41833338141441345),
 ('Reformist', 0.36586445569992065),
 ('Admon', 0.3658443093299866),
 ('Accomplishes', 0.3646325170993805),
 ('Likened', 0.36177918314933777),
 ('Ex-Lions', 0.36167073249816895),
 ('Osius', 0.3552625775337219),
 ('post-Marine', 0.35052719712257385)]

These seem like just noise. This is because of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with our given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

We can also get the vector for a given word by passing in the word as if we were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [14]:
wv['Texas']

array([ 2.5523796 ,  1.2822448 ,  0.41694877,  0.5538823 , -0.76307154,
       -1.4883628 ,  0.8776903 , -1.5675656 , -0.8796857 , -1.3575422 ,
       -0.816386  , -0.8863151 , -0.8770891 ,  0.40453276, -2.716295  ,
       -0.48600382, -1.2962158 ,  1.054702  ,  0.7552817 ,  1.1138533 ,
       -1.1696299 , -0.84171283,  1.7895814 , -0.13960691,  0.41298786,
        0.03221479,  1.8417407 , -1.5077775 ,  1.3027592 ,  0.49323556,
        0.23083155, -0.51468897, -1.8668255 , -2.51448   , -0.9757114 ,
        1.0984863 , -0.98975414,  1.0479026 ,  0.94862646, -0.7389775 ,
        0.1674252 ,  0.41704407, -0.11463501,  0.64166665,  0.1850271 ,
        1.5988101 ,  0.8745163 ,  1.3409345 ,  0.42044   , -0.00934597,
        0.11392583,  1.0030133 ,  1.2735151 ,  0.22078426, -1.154366  ,
       -1.5198121 ,  0.6739628 ,  0.84395736, -0.6703572 ,  3.7744086 ,
       -0.33799055, -0.09774343,  0.09180059, -1.6087893 ,  0.82125556,
       -2.201473  ,  0.8778543 ,  5.209053  ,  0.5855716 , -2.56

In [15]:
len(wv['Texas'])

100

Let's get all of the word vectors from the object at once. We can find these inside of `wv.vectors`.  Do this now in the cell below.  

In [16]:
wv.vectors

array([[ 4.48658466e-02,  4.79192466e-01, -7.02027559e-01, ...,
        -2.04666829e+00,  2.13234687e+00,  6.43534660e-01],
       [-9.69753861e-02,  6.62039399e-01, -1.23990488e+00, ...,
        -2.21684265e+00,  7.51824021e-01, -1.04964697e+00],
       [ 8.16364437e-02, -2.86136866e-01, -2.23528206e-01, ...,
        -1.04120278e+00,  9.04261649e-01, -1.31675148e+00],
       ...,
       [-2.15297136e-02, -6.29687533e-02, -7.79916649e-04, ...,
         1.02014393e-01, -8.79119039e-02,  6.08676113e-02],
       [-5.74546158e-02, -2.40545385e-02,  1.89367179e-02, ...,
         2.65812408e-02, -4.69285734e-02, -1.29967863e-02],
       [ 1.76124834e-02,  3.84184383e-02, -3.48509066e-02, ...,
         4.37084958e-03, -3.89629565e-02,  1.02934174e-01]], dtype=float32)

As a final exercise, let's try recreating the _'king' - 'man' + 'woman' = 'queen'_ example we've seen before. We can do this by using the `most_similar` function and putting the things we want added together inside of an array passed to the `positive` parameter, and the things we want subtracted as an array passed to the the `negative` parameter. 

Do this now in the cell below. 

In [17]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('queen', 0.6153455972671509),
 ('princess', 0.6070859432220459),
 ('reminiscent', 0.591543436050415),
 ('crown', 0.5824635028839111),
 ('Queen', 0.5729817748069763),
 ('supermodel', 0.5709661245346069),
 ('revival', 0.5619000196456909),
 ('villain', 0.5612452030181885),
 ('coin', 0.5479928851127625),
 ('du', 0.5472560524940491)]

As we can see from the output above, our model isn't perfect, but 'Queen' is still in the top 3, and with 'Princess' not too far behind. As we can see from the word in first place, 'reminiscent', our model is far from perfect. This is likely because we didn't give it too much training, or training data. However, for the small amount of training data it was given, the model still performs remarkably well! 

We'll see in the next lab that from a practical standpoint, one of the best things we can do for performance is to start by loading in the weights from an open-sourced model that has been trained for a very long time on a massive amount of data, such as the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless our text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, we learned how to train and use a Word2Vec model to created vectorized word embeddings!